Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [4]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
from html.parser import HTMLParser
# import HTMLParser

In [29]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [30]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [31]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [32]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
id       49159 non-null int64
label    31962 non-null float64
tweet    49159 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

In [33]:
HTMLParser.unescape('&lt', combine_df['tweet'])
HTMLParser.unescape('&gt', combine_df['tweet'])
HTMLParser.unescape('&amp', combine_df['tweet'])

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
5        [2/2] huge fan fare and big talking before the...
6         @user camping tomorrow @user @user @user @use...
7        the next school year is the year for exams.ð...
8        we won!!! love the land!!! #allin #cavs #champ...
9         @user @user welcome here !  i'm   it's so #gr...
10        â #ireland consumer price index (mom) climb...
11       we are so selfish. #orlando #standwithorlando ...
12       i get to see my daddy today!!   #80days #getti...
13       @user #cnn calls #michigan middle school 'buil...
14       no comment!  in #australia   #opkillingbay #se...
15       ouch...junior is angryð#got7 #junior #yugyo...
16       i am thankful for having a paner. #thankful #p.

2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [34]:
def proc(string):
    return re.sub('@user', '', string)

In [35]:
combine_df['tweet'].apply(proc)

0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i can't use cause th...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
5        [2/2] huge fan fare and big talking before the...
6                         camping tomorrow        dannyâ¦
7        the next school year is the year for exams.ð...
8        we won!!! love the land!!! #allin #cavs #champ...
9                    welcome here !  i'm   it's so #gr8 ! 
10        â #ireland consumer price index (mom) climb...
11       we are so selfish. #orlando #standwithorlando ...
12       i get to see my daddy today!!   #80days #getti...
13        #cnn calls #michigan middle school 'build the...
14       no comment!  in #australia   #opkillingbay #se...
15       ouch...junior is angryð#got7 #junior #yugyo...
16       i am thankful for having a paner. #thankful #p.

3. Изменим регистр твитов на нижний с помощью .lower()

In [36]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    return ret

In [37]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 250 ms, sys: 4.43 ms, total: 254 ms
Wall time: 265 ms


0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i can't use cause th...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
5        [2/2] huge fan fare and big talking before the...
6                         camping tomorrow        dannyâ¦
7        the next school year is the year for exams.ð...
8        we won!!! love the land!!! #allin #cavs #champ...
9                    welcome here !  i'm   it's so #gr8 ! 
10        â #ireland consumer price index (mom) climb...
11       we are so selfish. #orlando #standwithorlando ...
12       i get to see my daddy today!!   #80days #getti...
13        #cnn calls #michigan middle school 'build the...
14       no comment!  in #australia   #opkillingbay #se...
15       ouch...junior is angryð#got7 #junior #yugyo...
16       i am thankful for having a paner. #thankful #p.

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [38]:
def change_word(text, word_dict, mode=1):
    if mode == 1:
        for key in word_dict:
            text = re.sub(key, word_dict[key], text)
        return text
    elif mode == 2:
        ret = ''
        for word in text.split():
            ret = ret + change_word_2(word, word_dict) + ' '
        return ret.strip()
    else:
        return(text)

In [39]:
def change_word_2(word, word_dict):
    if word_dict.get(word):
        return(word_dict.get(word))
    else:
        return(word)

In [40]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    return ret

In [41]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 15.3 s, sys: 6.5 ms, total: 15.3 s
Wall time: 15.4 s


0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i cannot use cause t...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
5        [2/2] huge fan fare and big talking before the...
6                         camping tomorrow        dannyâ¦
7        the next school year is the year for exams.ð...
8        we won!!! love the land!!! #allin #cavs #champ...
9           welcome here !  I am   it has / it is so #g...
10        â #ireland consumer price index (mom) climb...
11       we are so selfish. #orlando #standwithorlando ...
12       i get to see my daddy today!!   #80days #getti...
13        #cnn calls #michigan middle school 'build the...
14       no comment!  in #australia   #opkillingbay #se...
15       ouch...junior is angryð#got7 #junior #yugyo...
16       i am thankful for having a paner. #thankful #p.

5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [42]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    return ret

In [43]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 27.5 s, sys: 36.8 ms, total: 27.5 s
Wall time: 27.8 s


0          when a fatheare is dysfyounctional and is so...
1          thanks foare #lyft careedit i cannot youse c...
2                                  bihday yoyouare majesty
3        #model   i love you take with you all the time...
4            factsgareinyouide: society now    #motivation
5        [2/2] hyougareine fan faaree and bigarein talk...
6                campingarein tomoareareow        dannyâ¦
7        the next school yeaare is the yeaare foare exa...
8        we won!!! love the land!!! #allin #cavs #champ...
9           welcome hearee !  I am   it has / it is so ...
10        â #iareeland consyoumeare pareI seee index ...
11       we aaree so selfish. #oarelando #standwithoare...
12       i gareinet to see my daddy today!!   #80days #...
13        #cnn calls #mI seehigareinan middle school 'b...
14       no comment!  in #ayoustarealia   #opkillingare...
15       oyouch...jyounioare is angareinareyð#garein...
16       i am thankfyoul foare havingarein a paneare. #.

6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [44]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    ret = change_word(ret, emoticon_dict, 2)
    return ret

In [45]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 28.5 s, sys: 88.8 ms, total: 28.6 s
Wall time: 29 s


0        when a fatheare is dysfyounctional and is so s...
1        thanks foare #lyft careedit i cannot youse cay...
2                                  bihday yoyouare majesty
3        #model i love you take with you all the time i...
4               factsgareinyouide: society now #motivation
5        [2/2] hyougareine fan faaree and bigarein talk...
6                       campingarein tomoareareow dannyâ¦
7        the next school yeaare is the yeaare foare exa...
8        we won!!! love the land!!! #allin #cavs #champ...
9        welcome hearee ! I am it has / it is so #garei...
10       â #iareeland consyoumeare pareI seee index (...
11       we aaree so selfish. #oarelando #standwithoare...
12       i gareinet to see my daddy today!! #80days #ga...
13       #cnn calls #mI seehigareinan middle school 'by...
14       no comment! in #ayoustarealia #opkillingareinb...
15       oyouch...jyounioare is angareinareyð#garein...
16       i am thankfyoul foare havingarein a paneare. #.

7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [46]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    ret = change_word(ret, emoticon_dict, 2)
    ret = re.sub(r'[^\w\s]', '', ret)
    return ret

In [47]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 28.9 s, sys: 61.6 ms, total: 29 s
Wall time: 29 s


0        when a fatheare is dysfyounctional and is so s...
1        thanks foare lyft careedit i cannot youse cayo...
2                                  bihday yoyouare majesty
3        model i love you take with you all the time in...
4                 factsgareinyouide society now motivation
5        22 hyougareine fan faaree and bigarein talking...
6                         campingarein tomoareareow dannyâ
7        the next school yeaare is the yeaare foare exa...
8        we won love the land allin cavs champions clev...
9        welcome hearee  I am it has  it is so gareinare8 
10       â iareeland consyoumeare pareI seee index mom ...
11       we aaree so selfish oarelando standwithoarelan...
12       i gareinet to see my daddy today 80days garein...
13       cnn calls mI seehigareinan middle school byoui...
14       no comment in ayoustarealia opkillingareinbay ...
15       oyouchjyounioare is angareinareyðgareinot7 jyo...
16       i am thankfyoul foare havingarein a paneare th.

8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [48]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    ret = change_word(ret, emoticon_dict, 2)
    ret = re.sub(r'[^\w\s]', '', ret)
    ret = re.sub(r'[^a-zA-Z0-9]', ' ', ret)
    return ret

In [49]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 29.2 s, sys: 31.7 ms, total: 29.2 s
Wall time: 29.4 s


0        when a fatheare is dysfyounctional and is so s...
1        thanks foare lyft careedit i cannot youse cayo...
2                                  bihday yoyouare majesty
3        model i love you take with you all the time in...
4                 factsgareinyouide society now motivation
5        22 hyougareine fan faaree and bigarein talking...
6                         campingarein tomoareareow danny 
7        the next school yeaare is the yeaare foare exa...
8        we won love the land allin cavs champions clev...
9        welcome hearee  I am it has  it is so gareinare8 
10         iareeland consyoumeare pareI seee index mom ...
11       we aaree so selfish oarelando standwithoarelan...
12       i gareinet to see my daddy today 80days garein...
13       cnn calls mI seehigareinan middle school byoui...
14       no comment in ayoustarealia opkillingareinbay ...
15       oyouchjyounioare is angareinarey gareinot7 jyo...
16       i am thankfyoul foare havingarein a paneare th.

9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [50]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    ret = change_word(ret, emoticon_dict, 2)
    ret = re.sub(r'[^\w\s]', '', ret)
    ret = re.sub(r'[^a-zA-Z0-9]', ' ', ret)
    ret = re.sub(r'[^a-zA-Z]', ' ', ret)
    return ret

In [51]:
%%time
combine_df['tweet'].apply(proc)

CPU times: user 30.1 s, sys: 51.9 ms, total: 30.2 s
Wall time: 30.4 s


0        when a fatheare is dysfyounctional and is so s...
1        thanks foare lyft careedit i cannot youse cayo...
2                                  bihday yoyouare majesty
3        model i love you take with you all the time in...
4                 factsgareinyouide society now motivation
5           hyougareine fan faaree and bigarein talking...
6                         campingarein tomoareareow danny 
7        the next school yeaare is the yeaare foare exa...
8        we won love the land allin cavs champions clev...
9        welcome hearee  I am it has  it is so gareinare  
10         iareeland consyoumeare pareI seee index mom ...
11       we aaree so selfish oarelando standwithoarelan...
12       i gareinet to see my daddy today   days garein...
13       cnn calls mI seehigareinan middle school byoui...
14       no comment in ayoustarealia opkillingareinbay ...
15       oyouchjyounioare is angareinarey gareinot  jyo...
16       i am thankfyoul foare havingarein a paneare th.

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [52]:
def proc(string):
    ret = re.sub('@user', '', string)
    ret = ret.lower()
    ret = change_word(ret, apostrophe_dict)
    ret = change_word(ret, short_word_dict)
    ret = change_word(ret, emoticon_dict, 2)
    ret = re.sub(r'[^\w\s]', '', ret)
    ret = re.sub(r'[^a-zA-Z0-9]', ' ', ret)
    ret = re.sub(r'[^a-zA-Z]', ' ', ret)
    ret = ' '.join([w for w in ret.split() if len(w)>1])
    return ret

In [53]:
%%time
combine_df['tweet'] = combine_df['tweet'].apply(proc)

CPU times: user 30.8 s, sys: 79.6 ms, total: 30.8 s
Wall time: 31.1 s


11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [58]:
# nltk.download()

In [54]:
def token(text):
    return nltk.word_tokenize(text)

In [57]:
%%time
combine_df['tweet_token'] = combine_df['tweet'].apply(token)

CPU times: user 17.5 s, sys: 89.3 ms, total: 17.6 s
Wall time: 19.2 s


12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [59]:
def drop_stop_words(tokens):
    ret = [w for w in tokens if len(w)>1]
    return [i for i  in ret if i not in nltk.corpus.stopwords.words('english')]

In [60]:
%%time
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].apply(drop_stop_words)

CPU times: user 2min 22s, sys: 14 s, total: 2min 36s
Wall time: 2min 38s


13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [62]:
def stremming_token(tokens):
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

In [63]:
%%time
combine_df['tweet_stemmed'] = combine_df['tweet_token_filtered'].apply(stremming_token)

CPU times: user 22.9 s, sys: 60.1 ms, total: 22.9 s
Wall time: 23.1 s


14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [64]:
def lemmatizer_token(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, nltk.corpus.wordnet.VERB) for word in tokens]

In [65]:
%%time
combine_df['tweet_lemmatized'] = combine_df['tweet_token_filtered'].apply(lemmatizer_token)

CPU times: user 8.59 s, sys: 160 ms, total: 8.75 s
Wall time: 8.98 s


15. Сохраним результат предобработки в pickle-файл.

In [67]:
import pickle

In [68]:
with open('result.pickle', 'wb') as f:
    pickle.dump(combine_df, f)